Data reading

In [7]:
import pandas as pd

In [8]:
from utils.FileUtils import IniFilesReader
from configConstants import EngineSimulationDataPath


iniFilesReader=IniFilesReader("files_path.ini")
dataPath=iniFilesReader.getProperty(EngineSimulationDataPath.ROOT)


In [9]:
names = ['engineNumber', 'cycleNumber',
         'altitude', 'mach', 'TRA',
         'T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr',
         'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']


trainDf : pd.DataFrame=pd.read_csv(dataPath[EngineSimulationDataPath.TRAIN_DATA_PATH]\
    ,header=None,delim_whitespace=" ",keep_default_na=False)
trainDf.columns=names

trainDf.head(10)

,engineNumber,cycleNumber,altitude,mach,TRA,T2,T24,T30,T50,P2,...,phi,NRf,NRc,BPR,farB,htBleed,Nf_dmd,PCNfR_dmd,W31,W32
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
5,1,6,-0.0043,-0.0001,100.0,518.67,642.10,1584.47,1398.37,14.62,...,521.68,2388.03,8132.85,8.4108,0.03,391,2388,100.0,38.98,23.3669
6,1,7,0.0010,0.0001,100.0,518.67,642.48,1592.32,1397.77,14.62,...,522.32,2388.03,8132.32,8.3974,0.03,392,2388,100.0,39.10,23.3774
7,1,8,-0.0034,0.0003,100.0,518.67,642.56,1582.96,1400.97,14.62,...,522.47,2388.03,8131.07,8.4076,0.03,391,2388,100.0,38.97,23.3106
8,1,9,0.0008,0.0001,100.0,518.67,642.12,1590.98,1394.80,14.62,...,521.79,2388.05,8125.69,8.3728,0.03,392,2388,100.0,39.05,23.4066
9,1,10,-0.0033,0.0001,100.0,518.67,641.71,1591.24,1400.46,14.62,...,521.79,2388.06,8129.38,8.4286,0.03,393,2388,100.0,38.95,23.4694


In [10]:

def add_remaining_useful_life(df):
    """
        Calculate the remaining usefull life
        The RUL is the result of the difference between
        the max cycle for the engine and the actual cycle
    """
    dfCopy=df.copy()
    max_df=trainDf.groupby('engineNumber').agg({'cycleNumber':'max'})
    # Get the total number of cycles for each unit
    max_cycle= max_df['cycleNumber']
    # Merge the max cycle back into the original frame
    dfCopy = dfCopy.merge(max_cycle.to_frame(name='max_cycle'), left_on='cycleNumber', right_index=True)
    # Calculate remaining useful life for each row
    remaining_useful_life = dfCopy["max_cycle"] - dfCopy["cycleNumber"]
    dfCopy["RUL"] = remaining_useful_life
    
    # drop max_cycle as it's no longer needed
    result_frame = dfCopy.drop("max_cycle", axis=1)
    return result_frame

In [11]:
add_remaining_useful_life(trainDf)

,engineNumber,cycleNumber,altitude,mach,TRA,T2,T24,T30,T50,P2,...,NRf,NRc,BPR,farB,htBleed,Nf_dmd,PCNfR_dmd,W31,W32,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
192,2,1,-0.0018,0.0006,100.0,518.67,641.89,1583.84,1391.28,14.62,...,2388.06,8137.72,8.3905,0.03,391,2388,100.0,38.94,23.4585,191
479,3,1,0.0008,0.0005,100.0,518.67,642.04,1584.20,1398.13,14.62,...,2388.00,8138.40,8.4207,0.03,391,2388,100.0,38.96,23.3205,191
658,4,1,0.0025,0.0005,100.0,518.67,642.60,1585.40,1402.85,14.62,...,2388.07,8131.50,8.4200,0.03,393,2388,100.0,39.04,23.3814,191
847,5,1,0.0031,0.0002,100.0,518.67,641.77,1583.59,1395.26,14.62,...,2388.00,8152.22,8.4102,0.03,393,2388,100.0,38.98,23.4468,191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19651,96,100,-0.0004,0.0001,100.0,518.67,642.38,1587.67,1404.62,14.62,...,2388.05,8131.77,8.4308,0.03,392,2388,100.0,39.06,23.3407,100
19987,97,100,0.0011,0.0003,100.0,518.67,642.19,1590.00,1405.37,14.62,...,2388.05,8137.96,8.4063,0.03,393,2388,100.0,38.97,23.3153,100
20189,98,100,-0.0029,0.0003,100.0,518.67,642.52,1595.92,1401.31,14.62,...,2388.05,8146.55,8.4331,0.03,392,2388,100.0,38.78,23.4594,100
20345,99,100,-0.0020,-0.0002,100.0,518.67,642.15,1591.06,1407.60,14.62,...,2388.12,8139.56,8.4152,0.03,392,2388,100.0,38.91,23.2626,100
